In [7]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager # 자동으로 크롬드라이버(가상브라우저) 파일을 다운로드해주는 라이브러리
from selenium.webdriver.chrome.service import Service # 다운로드된 크롬드라이버 파일을 연결하기 위해 활용

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time, csv

In [12]:
# 일단 'https://chromedriver.chromium.org/downloads'에서 자신의 크롬 버젼에 맞는 드라이버 다운 후
# 윈도우즈는 C드라이브에, 맥북은 Documents에 저장

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# Mac은 '/Users/***컴퓨터계정이름***/Documents/chromedriver'
# Windows는 'C:/chromedriver.exe'

url = 'https://www.airkorea.or.kr/web/pmRelay?itemCode=11008&pMENU_NO=109'

# 웹사이트 열기
driver.get(url)
driver.implicitly_wait(10)
# 로딩이 끝날 때까지 10초까지 기다림

driver.find_element(By.CSS_SELECTOR,'#strDateDiv2').click() # 일평균 클릭
time.sleep(1)

current_file_index = 0

df_final = pd.DataFrame(columns = ["시", "연도", "월", "지점ID", "지점"])


for y in range(1,9):
    year_option = driver.find_element(By.CSS_SELECTOR, f'#searchDate_yyyy > option:nth-child({y})')
    year = year_option.text
    year_option.click()
    time.sleep(1)

    for m in range(13,1, -1):
        if y == 1 and m > 11:
            continue
        month_option = driver.find_element(By.CSS_SELECTOR, f'#searchDate_mm > option:nth-child({m})')
        month = month_option.text
        month_option.click()
        time.sleep(1)

        for d in range(1,18):
            district_option = driver.find_element(By.CSS_SELECTOR,f'#district > option:nth-child({d})')
            district = district_option.text
            district_option.click()
            time.sleep(1)

            driver.find_element(By.CSS_SELECTOR,'#cont_body > form > div > div > a:nth-child(1)').click() # 검색 클릭
            wait = WebDriverWait(driver, 10)
            time.sleep(20)
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            t = soup.select("#tablefix1")
            table = pd.read_html(str(t))[0]


            df = pd.DataFrame(table)
            df["연도"] = int(year)
            df["월"] = int(month)
            df["시"] = district

            df_final = pd.concat([df_final, df])

            current_file_index += 1
            

KeyboardInterrupt: 